# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: NIRSpec full cube model

Michael Reefe

This example notebook provides a tutorial on how to run LOKI on a NIRSpec IFU cube, doing a full fit of each spaxel.

In this example, we'll utilize the multi-processing capabilities of the code.

In [7]:
using Distributed 
procs = addprocs(Sys.CPU_THREADS, exeflags="--heap-size-hint=4G")

@everywhere begin 
    using Pkg
    Pkg.activate(dirname(@__DIR__))
    Pkg.instantiate()
    Pkg.precompile()
    using Loki
    using Unitful 
end

      From worker 3:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


      From worker 2:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`
      From worker 5:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`
      From worker 4:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Now we want to load in our data. For this example, we'll be using the data for II Zw 96 in the G235H/F170LP grating/filter combination, which is located in the same folder as this notebook. The JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [8]:
# The redshift of the target object: II Zw 96
z = 0.036098
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs = from_fits(["jw01328-o036_t029_nirspec_g235h-f170lp_s3d.fits.gz"], z);

[ Info: Initializing DataCube struct from jw01328-o036_t029_nirspec_g235h-f170lp_s3d.fits.gz


Next, we create some variables that we will use later. We will be fitting data from one grating/filter combo, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [9]:
channel = :G235H_F170LP
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_$(channel)_full_model"

"IIZw96_G235H_F170LP_full_model"

Before fitting, we want to do some pre-processing on the data. We want to convert the data to the rest-frame, mast out / interpolate any bad pixels, and replace the JWST pipeline-generated errors with some more realistic ones.  All of this is achieved in the next block of code.

In [10]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)

    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end

[ Info: Initializing DataCube struct from IIZw96.channelG235H_F170LP.rest_frame.fits


Observation(Dict{Any, DataCube}(:G235H_F170LP => DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[1.6023561155563175 μm, 1.6027383187731377 μm, 1.6031206131552722 μm, 1.6035029987244664 μm, 1.6038854755024705 μm, 1.6042680435110404 μm, 1.6046507027719368 μm, 1.605033453306926 μm, 1.6054162951377788 μm, 1.6057992282862719 μm  …  3.0530264517811982 μm, 3.0537546772484205 μm, 3.054483076416179 μm, 3.055211649325906 μm, 3.0559403960190425 μm, 3.056669316537041 μm, 3.0573984109213628 μm, 3.0581276792134795 μm, 3.0588571214548734 μm, 3.059586737687035 μm], Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}[NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ … NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹; NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ NaN erg Hz⁻¹ cm⁻² s⁻¹ sr⁻¹ 

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. Note specifically for NIRSpec data, we want to enable the option "nirspec_mask_chip_gaps".  This will make sure that when fitting any integrated regions larger than a single spaxel (i.e. apertures, voronoi bins, or the initial integrated fit over the whole FOV) the chip gap regions will be fully masked out.  This is necessary because the wavelengths of the chip gaps vary based on the position in the IFU, so when making integrated spectra, in regions where some spaxels are masked and others aren't, it can create artifical dips in the continuum.  The "nirspec_mask_chip_gaps" options masks out the maximum extent of the chip gaps over all of the IFU positions (but again, only when fitting regions larger than a single spaxel).

Note: we're doing the whole shebang for this example, so it may take a few hours to complete!

In [11]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(
    obs.channels[channel], 
    obs.z, 
    run_name; 
    nirspec_mask_chip_gaps=true,
    parallel=true, 
    plot_spaxels=:pyplot, 
    plot_maps=true, 
    save_fits=true,
    silicate_absorption="d+",
    extinction_screen=true, 
    use_pah_templates=true,
    fit_sil_emission=false, 
    fit_stellar_continuum=true, 
    save_full_model=true, 
    map_snr_thresh=3., 
    subtract_cubic_spline=true,
    stellar_template_type="stars",   # => use single-star templates rather than SSPs (more flexible)
)

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter)

[ Info: Preparing output directories
┌ Warning: Setting ssp_regularize=0.0 since stellar_template_type is not "ssp"
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/cubefit.jl:335
┌ Warning: There are no PAH features within the input wavelength range of the spectrum. The use_pah_templates option will be disabled.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/create_params.jl:540
[ Info: Loading pre-generated stellar templates from binary file
┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR IIZw96_G235H_F170LP_full_model ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     4
│ Threads per process:  1
└ ------------------------
[ Info: ===> Preparing output data structures... <===
[ Info: ===> Initial fit to the sum of all spaxels is being skipped, either because it has already

Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


[ Info: ===> Beginning individual spaxel fitting... <===


Progress:  41%|████████████████▉                        |  ETA: 0:49:24

      From worker 5:	┌ Warning: LM Solver is stuck on the initial state for the line fit of spaxel CartesianIndex(21, 43). Jittering starting params...
      From worker 5:	└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/fitting.jl:29


Progress: 100%|█████████████████████████████████████████| Time: 1:23:13


[ Info: ===> Generating parameter maps and model cubes... <===


Progress: 100%|███████████████████████████| Time: 0:05:43 ( 0.12  s/it)


[ Info: ===> Plotting parameter maps... <===
┌ Warning: You have requested combined plots for the lines [:OII_3727A, :OII_3729A], but at least one of these lines was not fit!
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/output.jl:665
[ Info: ===> Writing FITS outputs... <===
[ Info: ===> Writing CSV tables... <=== 
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! ##################################
└ #############################################################################


(CubeFitter{Float64, Int64, Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}(DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[1.6023561155563175 μm, 1.6027383187731377 μm, 1.6031206131552722 μm, 1.6035029987244664 μm, 1.6038854755024705 μm, 1.6042680435110404 μm, 1.6046507027719368 μm, 1.605033453306926 μm, 1.6054162951377788 μm, 1.6057992282862719 μm  …  3.0530264517811982 μm, 3.0537546772484205 μm, 3.054483076416179 μm, 3.055211649325906 μm, 3.0559403960190425 μm, 3.056669316537041 μm, 3.0573984109213628 μm, 3.0581276792134795 μm, 3.0588571214548734 μm, 3.059586737687035 μm], Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹

This makes sure that we terminate all of the parallel worker processes that we started:

In [12]:
rmprocs(procs)

Task (done) @0x00000003ab188650

And the results can be found in the "output_[run_name]" directory, just like the other examples!  Here is an example of the initial fit to the integrated spectrum over the whole FOV:

![](./IIZw96_initial_fit.png)

The orange line shows the final model.  The decomposed components of the model consist of:
- Thermal dust continuum, in gray
- Stellar continuum, in pink
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

The gray band(s) shows a masked out part of the spectrum -- this is due to NIRSpec's chip gap.

Notice that many emission lines aren't being fit! This is because we don't have them in our "lines.toml" config file.  You will need to add these manually if you want to include them.  Just make sure to follow the format of the other line entries in the file.

And here are the parameter maps for the line flux, velocity shift, and FWHM for Paschen alpha:

![](./IIZw96.Paalpha.flux.png)
![](./IIZw96.Paalpha.voff.png)
![](./IIZw96.Paalpha.fwhm.png)

And a map of the stellar mass:

![](./IIZw96.stellar_mass.png)
